
**Nama** : Zidan Muhammad Ikvan,
**Cohort ID** : MC404D5Y0059,
**Email** : zidanikvan@gmail.com

## Import Library/Package

In [1]:
!pip install gensim
!pip install numpy==1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.1 MB/s eta 0:00:00


In [2]:
import re
import string
import nltk
import numpy as np
import pandas as pd
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Load dataset

In [3]:
df = pd.read_csv("/content/shopee_reviews_final.csv")


print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10035 entries, 0 to 10034
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   userName              10035 non-null  object 
 1   score                 10035 non-null  int64  
 2   content               10034 non-null  object 
 3   reviewId              1715 non-null   object 
 4   userImage             1715 non-null   object 
 5   thumbsUpCount         1715 non-null   float64
 6   reviewCreatedVersion  1340 non-null   object 
 7   at                    1715 non-null   object 
 8   replyContent          1506 non-null   object 
 9   repliedAt             1506 non-null   object 
 10  appVersion            1340 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 862.5+ KB
None
                 userName  score  \
0  Eka Simamora## Channel      5   
1       Sasmito Alfa (Al)      5   
2               Tuti Asih      5   
3           Zai

In [4]:
df = df.dropna(subset=['content'])

In [7]:
# Menyisakan hanya kolom yang diperlukan: nama user, skor, dan isi ulasan
df = df[['userName', 'score', 'content']]

# Menyimpan ulang versi bersih
df.to_csv("shopee_reviews_cleaned.csv", index=False)
print(df.head())
print(f"Jumlah data akhir: {len(df)}")

                 userName  score  \
0  Eka Simamora## Channel      5   
1       Sasmito Alfa (Al)      5   
2               Tuti Asih      5   
3           Zainal Arifin      5   
4               mas yanto      5   

                                             content  
0                        sangat bermanfaat dan bagus  
1  sudah ada peningkatan.. semenjak saya komplain...  
2                      Puas dan happy bersama shopee  
3               produknya banyak harganya terjangkau  
4                                              bagus  
Jumlah data akhir: 10034


In [8]:
df = pd.read_csv("/content/shopee_reviews_cleaned.csv")


print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10034 entries, 0 to 10033
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  10034 non-null  object
 1   score     10034 non-null  int64 
 2   content   10034 non-null  object
dtypes: int64(1), object(2)
memory usage: 235.3+ KB
None
                 userName  score  \
0  Eka Simamora## Channel      5   
1       Sasmito Alfa (Al)      5   
2               Tuti Asih      5   
3           Zainal Arifin      5   
4               mas yanto      5   

                                             content  
0                        sangat bermanfaat dan bagus  
1  sudah ada peningkatan.. semenjak saya komplain...  
2                      Puas dan happy bersama shopee  
3               produknya banyak harganya terjangkau  
4                                              bagus  


In [9]:
print("Missing values:\n", df.isnull().sum())

Missing values:
 userName    0
score       0
content     0
dtype: int64


In [10]:
print("jumlah duplikat", df.duplicated().sum())

jumlah duplikat 0


## Preprocessing Text

In [11]:
# Load stopwords
stop_words = set(stopwords.words('indonesian'))

# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi preprocessing
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    # Hapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Hapus emoji dan karakter aneh
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Hapus whitespace berlebih
    text = text.strip()
    # Tokenisasi dan hapus stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    # Stemming
    text = ' '.join([stemmer.stem(word) for word in words])
    return text

# Menerapkan ke kolom content
df['clean_content'] = df['content'].apply(preprocess_text)

# Cek hasil
print(df[['content', 'clean_content']].head())

                                             content  \
0                        sangat bermanfaat dan bagus   
1  sudah ada peningkatan.. semenjak saya komplain...   
2                      Puas dan happy bersama shopee   
3               produknya banyak harganya terjangkau   
4                                              bagus   

                                       clean_content  
0                                      manfaat bagus  
1  tingkat semenjak komplain kantor pusat bagus k...  
2                                  puas happy shopee  
3                               produk harga jangkau  
4                                              bagus  


## Pelabelan

In [12]:
# Menghapus data dengan skor 3
df = df[df['score'] != 3]

# Menambahkan kolom label
df['label'] = df['score'].apply(lambda x: 'positif' if x > 3 else 'negatif')

# Cek distribusi label
print(df['label'].value_counts())

label
positif    7389
negatif    2213
Name: count, dtype: int64


<ipython-input-12-5bc8dd0a9c4f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['score'].apply(lambda x: 'positif' if x > 3 else 'negatif')


## Data Splitting dan Ekstraksi Fitur dengan TF-IDF, Word2Vec

In [13]:
# TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_content'])
y = df['label']

# Split data 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Split data 70:30
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [14]:
# Tokenisasi
tokens = [row.split() for row in df['clean_content']]

# Latih Word2Vec
w2v_model = Word2Vec(sentences=tokens, vector_size=100, window=5, min_count=2, workers=4, sg=1)

In [15]:
def avg_vector(text, model, vector_size=100):
    words = text.split()
    valid_words = [w for w in words if w in model.wv]
    if not valid_words:
        return np.zeros(vector_size)
    return np.mean([model.wv[w] for w in valid_words], axis=0)

X_w2v = np.array([avg_vector(text, w2v_model, 100) for text in df['clean_content']])

# Data Split
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, df['label'], test_size=0.2, random_state=42)

## Modelling

In [16]:
model_svm = LinearSVC()
model_svm.fit(X_train, y_train)

# Predict
y_pred_svm = model_svm.predict(X_test)

# Evaluasi
print("Akurasi testing SVM:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

y_train_pred = model_svm.predict(X_train)

# Evaluasi akurasi training
print("Akurasi Training SVM:", accuracy_score(y_train, y_train_pred))

Akurasi testing SVM: 0.9125455491931286
              precision    recall  f1-score   support

     negatif       0.83      0.77      0.80       443
     positif       0.93      0.95      0.94      1478

    accuracy                           0.91      1921
   macro avg       0.88      0.86      0.87      1921
weighted avg       0.91      0.91      0.91      1921

Akurasi Training SVM: 0.9714880874886083


In [17]:
# Inisialisasi model
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Training (gunakan X_train_1 dan y_train_1 dari split 70:30)
model_rf.fit(X_train_1, y_train_1)

# Prediksi (gunakan X_test_1 dari split 70:30)
y_pred_rf = model_rf.predict(X_test_1)
y_train_pred_rf = model_rf.predict(X_train_1)

# Evaluasi (gunakan y_test_1 dan y_train_1 dari split 70:30)
print("\n=== Hasil dengan Pembagian 70/30 ===")
print("Akurasi Training RF:", accuracy_score(y_train_1, y_train_pred_rf))
print("Akurasi Testing RF:", accuracy_score(y_test_1, y_pred_rf))
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_1, y_pred_rf))


=== Hasil dengan Pembagian 70/30 ===
Akurasi Training RF: 0.9953875911322719
Akurasi Testing RF: 0.8986463033668864

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.79      0.76      0.78       664
     positif       0.93      0.94      0.93      2217

    accuracy                           0.90      2881
   macro avg       0.86      0.85      0.86      2881
weighted avg       0.90      0.90      0.90      2881



In [18]:
# Modelling (Random Forest)
model_rf_w2v = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf_w2v.fit(X_train_w2v, y_train_w2v)

# Evaluasi
y_pred_rf_w2v = model_rf_w2v.predict(X_test_w2v)
y_train_pred_rf_w2v = model_rf_w2v.predict(X_train_w2v)

print("Akurasi Training RF + Word2Vec:", accuracy_score(y_train_w2v, y_train_pred_rf_w2v))
print("Akurasi Testing RF + Word2Vec:", accuracy_score(y_test_w2v, y_pred_rf_w2v))
print(classification_report(y_test_w2v, y_pred_rf_w2v))

Akurasi Training RF + Word2Vec: 0.9954433016534305
Akurasi Testing RF + Word2Vec: 0.8833940655908381
              precision    recall  f1-score   support

     negatif       0.74      0.78      0.76       446
     positif       0.93      0.92      0.92      1475

    accuracy                           0.88      1921
   macro avg       0.83      0.85      0.84      1921
weighted avg       0.89      0.88      0.88      1921



In [19]:
# Naive bayes
model_nb = MultinomialNB()
model_nb.fit(X_train, y_train)

y_pred_nb = model_nb.predict(X_test)
y_train_pred_nb = model_nb.predict(X_train)

print("Akurasi Training NB + TF-IDF:", accuracy_score(y_train, y_train_pred_nb))
print("Akurasi Testing NB + TF-IDF:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

Akurasi Training NB + TF-IDF: 0.9191511521937248
Akurasi Testing NB + TF-IDF: 0.9000520562207184
              precision    recall  f1-score   support

     negatif       0.86      0.67      0.76       443
     positif       0.91      0.97      0.94      1478

    accuracy                           0.90      1921
   macro avg       0.89      0.82      0.85      1921
weighted avg       0.90      0.90      0.90      1921



## Inference

In [20]:
# Contoh inferensi pada data baru
sample_review = ["Shopee adalah aplikasi belanja yang sangat baik!", "Produk sangat buruk, tidak sesuai ekspektasi."]
sample_tfidf = tfidf.transform(sample_review)
predictions = model_nb.predict(sample_tfidf)

# Output hasil inferensi
print("Hasil Prediksi:", predictions)

Hasil Prediksi: ['positif' 'positif']


In [21]:
# Contoh inferensi pada data baru
sample_review = ["Shopee adalah aplikasi belanja yang sangat baik!", "Produk sangat buruk, tidak sesuai ekspektasi."]
sample_tfidf = tfidf.transform(sample_review)
predictions = model_rf.predict(sample_tfidf)

# Output hasil inferensi
print("Hasil Prediksi:", predictions)

Hasil Prediksi: ['positif' 'negatif']


In [22]:
# Contoh inferensi pada data baru
sample_review = ["Shopee adalah aplikasi belanja yang sangat baik!", "Produk sangat buruk, tidak sesuai ekspektasi."]
sample_tfidf = tfidf.transform(sample_review)
predictions = model_svm.predict(sample_tfidf)

# Output hasil inferensi )
print("Hasil Prediksi:", predictions)

Hasil Prediksi: ['positif' 'positif']


In [23]:
# Contoh inferensi pada data baru
sample_review = ["Shopee adalah aplikasi belanja yang sangat baik!", "Produk sangat buruk, tidak sesuai ekspektasi."]

# Preprocessing teks (sesuai clean_content)
sample_clean = [text.lower().replace(".", "").replace("!", "") for text in sample_review]

# Konversi teks ke vektor rata-rata Word2Vec
sample_vectors = np.array([avg_vector(text, w2v_model, 100) for text in sample_clean])

# Prediksi dengan model Random Forest yang sudah dilatih
predictions = model_rf_w2v.predict(sample_vectors)

print("Hasil Prediksi:", predictions)

Hasil Prediksi: ['positif' 'positif']


## Kesimpulan

- Model terbaik secara keseluruhan dalam hal akurasi dan konsistensi prediksi pada data baru adalah Random Forest dengan fitur TF-IDF, karena berhasil mengenali kelas negatif dengan lebih akurat pada inferensi nyata.

- SVM dengan TF-IDF menunjukkan performa tinggi pada metrik akurasi dan f1-score, tetapi agak kurang dalam mengenali sentimen negatif.

- Model berbasis Word2Vec juga menunjukkan hasil yang kompetitif, namun tidak memberikan keunggulan signifikan dibandingkan pendekatan TF-IDF.

- Kelemahan utama dari semua model adalah dalam mendeteksi kelas minoritas (negatif), sehingga di proyek yang akan datang perlu dilakukan penyeimbangan data atau penggunaan teknik peningkatan minoritas seperti SMOTE.